In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tqdm

square_size = 1.75 # 正方形の1辺のサイズ[cm]
mov_file = 'calibration.MOV'

In [3]:
pattern_size = (7, 7)  # 交差ポイントの数
pattern_points = np.zeros( (np.prod(pattern_size), 3), np.float32 ) #チェスボード（X,Y,Z）座標の指定 (Z=0)
pattern_points[:,:2] = np.indices(pattern_size).T.reshape(-1, 2)
pattern_points *= square_size
objpoints = []
imgpoints = []

cap = cv2.VideoCapture(mov_file)
with tqdm.tqdm(total=int(cap.get(cv2.CAP_PROP_FRAME_COUNT)), ncols=100) as bar:
    for ii in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
        bar.update(1)
        ret, img = cap.read()
        if not ret:
            break
        if (ii % int(cap.get(cv2.CAP_PROP_FPS)/2)) <= 0:
            height = img.shape[0]
            width = img.shape[1]

            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            ret, corner = cv2.findChessboardCorners(gray, pattern_size)
            # コーナーがあれば
            if ret == True:
                bar.set_description("[Detected %d]" % (len(objpoints)+1))
                term = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_COUNT, 30, 0.1)
                cv2.cornerSubPix(gray, corner, (5,5), (-1,-1), term)
                imgpoints.append(corner.reshape(-1, 2))   #appendメソッド：リストの最後に因数のオブジェクトを追加
                objpoints.append(pattern_points)
        
    
cap.release()

print("calculating camera parameter...")
# 内部パラメータを計算
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

print("RMS = ", ret)
print("mtx = \n", mtx)
print("dist = ", dist.ravel())

[Detected 100]:  99%|█████████████████████████████████████████▋| 1531/1541 [00:07<00:00, 207.75it/s]


calculating camera parameter...
RMS =  0.9886348928284457
mtx = 
 [[1.16900207e+03 0.00000000e+00 1.00991928e+03]
 [0.00000000e+00 1.18859010e+03 6.43856458e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
dist =  [-4.15195638e-01  1.94341693e-01  1.27695775e-03  3.27689771e-04
 -4.62271597e-02]
